# Better Transformer (BT)

PyTorch 1.12 sürümünün bir parçası olarak Better Transformer (BT) tanıtılmaktadır.Better Transformer, CPU ve GPU'da yüksek performansla Transformer modellerinin dağıtımını hızlandırmak için üretime hazır bir hızlı yoldur. Fastpath özelliği, doğrudan PyTorch core nn.module veya torchtext tabanlı modeller için şeffaf bir şekilde çalışır.TransformerEncoder, TransformerEncoderLayer ve MultiHeadAttention kullanarak bu işlemleri yapar.

## Ön Hazırlıklar

Better Transformer iki tür hızlandırma sunar:


1.   Genel yürütme verimliliğini artırmak amacıyla CPU ve GPU için yerel multihead attention (MHA) mekanizmasını uygular.
2.   NLP çıkarımında seyrekliği kullanmak. Değişken girdi uzunlukları nedeniyle, girdi belirteçleri, işlemenin atlanabileceği çok sayıda dolgu belirteci içerebilir ve bu da önemli hızlanmalar sağlar.

Fastpath execution bazı kriterlere tabidir. En önemlisi, modelin çıkarım modunda çalıştırılması ve gradyan bilgilerini toplamayan( örneğin with torch.no_grad modu ) giriş tensörleri üzerinde çalışması gerekir.



## Şunların testini ve tespitini yapacağız.



*   Önceden eğitilmiş modelleri yükleyin (Better Transformer olmadan oluşturulan 1.12 version ve öncesi)
*   BT fastpath yolu olan ve olmayan CPU'da çıkarım çalıştırma ve kıyaslama karşılaştırması (yalnızca yerel MHA)
*   BT fastpath yolu olan ve olmayan yapılandırılabilir) CİHAZ üzerinde çıkarım çalıştırın ve kıyaslama yapın (yalnızca yerel MHA)
*   Sparsity /Seyreklik desteğini etkinleştir
*   BT fastpath yolu olan ve olmayan (yerel MHA + seyreklik) (yapılandırılabilir) CİHAZ üzerinde çıkarım çalıştırın ve kıyaslama yapın

Daha fazla bilgi için : https://pytorch.org/blog/a-better-transformer-for-fast-transformer-encoder-inference//


## 1.Önceden eğitilmiş modelleri yükleyin.

XLM-R modelini, torchtext.models içerisindeki yönergeleri takip ederek önceden tanımlı torchtext modellerinden indiriyoruz. Cihazı ayrıca hızlandırıcı üzerinde testler yapacak şekilde ayarladık. (Ortamınız için uygun şekilde GPU yürütmeyi etkinleştirin.)

In [2]:
import torch
import torch.nn as nn

print(f"torch version: {torch.__version__}")

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"torch cuda available: {torch.cuda.is_available()}")

import torch, torchtext
from torchtext.models import RobertaClassificationHead
from torchtext.functional import to_tensor
xlmr_large = torchtext.models.XLMR_LARGE_ENCODER
classifier_head = torchtext.models.RobertaClassificationHead(num_classes=2, input_dim = 1024)
model = xlmr_large.get_model(head=classifier_head)
transform = xlmr_large.transform()

torch version: 1.13.1+cu116
torch cuda available: True


## 2.Data Setup

İki tür girdi oluşturduk: küçük bir girdi grubu ve seyrekliğe sahip büyük bir girdi grubu.

In [3]:
small_input_batch = [
               "Hello world",
               "How are you!"
]
big_input_batch = [
               "Hello world",
               "How are you!",
               """`Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.`

It was in July, 1805, and the speaker was the well-known Anna
Pavlovna Scherer, maid of honor and favorite of the Empress Marya
Fedorovna. With these words she greeted Prince Vasili Kuragin, a man
of high rank and importance, who was the first to arrive at her
reception. Anna Pavlovna had had a cough for some days. She was, as
she said, suffering from la grippe; grippe being then a new word in
St. Petersburg, used only by the elite."""
]

Ardından, küçük veya büyük girdi yığınını seçer, girdileri önceden işler ve modeli test ederiz.

In [4]:
input_batch=big_input_batch

model_input = to_tensor(transform(input_batch), padding_value=1)
output = model(model_input)
output.shape

torch.Size([3, 2])

Son olarak, kıyaslama yineleme sayısını belirledik

In [5]:
ITERATIONS=10

## 3.BT fastpath yolu olan ve olmayan CPU'da çıkarım çalıştırma ve kıyaslama karşılaştırması (yalnızca yerel MHA)

İlk çalıştırma, geleneksel yürütmeyi kullanır.

İkinci çalıştırma, modeli model.eval() kullanarak çıkarım moduna getirerek BT fastpath yürütmesini etkinleştirir ve torch.no_grad() ile gradyan toplamayı devre dışı bırakır.

Model CPU üzerinde yürütülürken (büyüklüğü CPU modeline bağlı olacaktır) bir gelişme görebilirsiniz. Fastpath profilinin yerel TransformerEncoderLayer uygulamasında yürütme süresinin çoğunu transformer_encoder_layer_fwd gösterdiğine dikkat edin.

In [ ]:
print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
  for i in range(ITERATIONS):
    output = model(model_input)
print(prof)

model.eval()

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
  with torch.no_grad():
    for i in range(ITERATIONS):
      output = model(model_input)
print(prof)

slow path:


## 4.BT fastpath yolu olan ve olmayan (yapılandırılabilir) cihaz üzerinde çalıştırma ve kıyaslama çıkarımı (yalnızca yerel MHA)

 BT sparsity ayarlarını kontrol edeceğiz.

In [6]:
model.encoder.transformer.layers.enable_nested_tensor

True

Şimdi BT sparsity ayarlarını devre dışı bırakalım.

In [7]:
model.encoder.transformer.layers.enable_nested_tensor=False

Modeli cihaz üzerinde çalıştırıyoruz ve cihaz üzerinde yerel MHA yürütmesi için profil bilgilerini topluyoruz:

İlk çalıştırma, geleneksel yürütmeyi kullanır.

İkinci çalıştırma, modeli model.eval() kullanarak çıkarım moduna getirerek BT fastpath yürütmesini etkinleştirir ve torch.no_grad() ile gradyan toplamayı devre dışı bırakır.

Bir GPU'da çalıştırırken, özellikle küçük girdi grubu ayarı için önemli bir hızlanma görmelisiniz.

In [ ]:
model.to(DEVICE)
model_input = model_input.to(DEVICE)

print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  for i in range(ITERATIONS):
    output = model(model_input)
print(prof)

model.eval()

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  with torch.no_grad():
    for i in range(ITERATIONS):
      output = model(model_input)
print(prof)

## 5.BT fastpath yolu olan veya olmayan (yerel MHA + BT sparsity) (yapılandırılabilir) cihaz üzerinde çıkarımı

BT sparsity desteğini etkinleştiriyoruz.

In [8]:
model.encoder.transformer.layers.enable_nested_tensor = True

Modeli cihaz üzerinde çalıştırıyoruz ve cihaz üzerinde yerel MHA ve sparsity desteği yürütmesi için profil bilgilerini topluyoruz:

İlk çalıştırma, geleneksel yürütmeyi kullanır.

İkinci çalıştırma, modeli model.eval() kullanarak çıkarım moduna getirerek BT hızlı yol yürütmesini etkinleştirir ve torch.no_grad() ile gradyan toplamayı devre dışı bırakır.

Bir GPU'da çalıştırırken, özellikle seyreklik içeren büyük toplu girdi ayarı için önemli bir hızlanma görmelisiniz

In [ ]:
model.to(DEVICE)
model_input = model_input.to(DEVICE)

print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  for i in range(ITERATIONS):
    output = model(model_input)
print(prof)

model.eval()

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  with torch.no_grad():
    for i in range(ITERATIONS):
      output = model(model_input)
print(prof)

## Özet

Bu eğitimde, Transformer Encoder modelleri için PyTorch çekirdeği Better Transformer desteğini kullanarak torch metninde Better Transformer hızlı yol yürütme ile hızlı transformatör çıkarımını tanıttık. Better Transformer'ın kullanımını, BT fastpath yol yürütmesinin kullanıma sunulmasından önce eğitilmiş modellerle gösterdik. Hem BT fastpath yürütme modlarının, yerel MHA yürütmenin hem de BT sparsity hızlandırmasının kullanımını gösterdik ve kıyasladık.